In [1]:
import tensorflow as tf
import numpy as np

/Users/tommy/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [126]:
data_set = []
label_set = []
for i in range(1,1001):
    data = [1.0/i]
    label = [1.0/i]
    data_set.append(data)
    label_set.append(label)
data_set=np.array(data_set)
label_set=np.array(label_set)

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

data_set,label_set = unison_shuffled_copies(data_set,label_set)
print(data_set[0:2])

[[0.00154083]
 [0.00699301]]


In [136]:

class Model(object):
    def __init__(self):
        self.input = tf.placeholder(tf.float32,shape=[None,1],name='input')
        self.label = tf.placeholder(tf.float32,shape=[None,1],name='label')
        self.global_step = tf.Variable(0, name='global_step',trainable=False)
        self.keep_prob = tf.placeholder(tf.float32,name='keep_prob')
        
    def denselayer_with_dropout(self, input_, out_dim , keep_prob):
        '''
        dense = tf.layers.dense(inputs=input_, units= out_dim , use_bias = False,
                                activation=tf.nn.relu)#tf.nn.relu
        return dense
        '''
        w = tf.Variable(tf.random_normal([]), name='weight')
        b = tf.Variable(tf.random_normal([]), name='bias')
        nonlin_model = tf.add(tf.multiply(tf.tanh(input_), w), b)
        return nonlin_model
    
    def inference(self):
        dense_out = self.denselayer_with_dropout(self.input,1,1)
        return dense_out
    
    def build_optimizer(self,loss_val,trainable_var,learning_rate):
        optimizer = tf.train.AdamOptimizer(learning_rate) 
        grads = optimizer.compute_gradients(loss_val, var_list=trainable_var)
        return optimizer.apply_gradients(grads,global_step=self.global_step)
        
    def build_train(self,learning_rate):
        raw_output = self.inference()
        rawloss = tf.losses.mean_squared_error(self.label,raw_output)
        loss = tf.reduce_mean(rawloss)
        trainable_var = tf.trainable_variables()
        train_op = self.build_optimizer(loss,trainable_var,learning_rate)
        return train_op,loss,raw_output
    
    def before_session_initialization(self,learning_rate):
        self.train_op,self.loss_op, self.raw_out = self.build_train(
            learning_rate=learning_rate)
        
    def after_session_initialization(self,sess):
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        self.train_writer = tf.summary.FileWriter('./', sess.graph)
        
    def train(self,sess,data,label):
        for i in range(10):
            batch_data = data[i*100:(i+1)*100]
            batch_label = label[i*100:(i+1)*100]
            feed_dict = {self.input: batch_data, self.label: batch_label}
            _, train_loss,out= sess.run([self.train_op,self.loss_op,self.raw_out], feed_dict=feed_dict)
            print(train_loss)
            #print(out)
            #print(batch_label)
            
    def test(self,sess,data):
        inp = [data]
        print(inp)
        feed_dict = {self.input: [data]}
        out= sess.run([self.raw_out], feed_dict=feed_dict)
        print(out)

tf.reset_default_graph()
model = Model()
model.before_session_initialization(learning_rate=1)
sess = tf.InteractiveSession()
model.after_session_initialization(sess)
model.train(sess,data_set,label_set)
model.test(sess,data_set[200])
print(label_set[200])
sess.close()

0.11946488
0.44358623
0.09755578
0.034890607
0.20631625
0.15162489
0.019592391
0.03032659
0.103214525
0.10095323
[array([0.0212766])]
[array([[0.21093656]], dtype=float32)]
[0.0212766]


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tqdm
import matplotlib.pyplot as plt
from omnicomm_data.test_data import get_model
import os
from sklearn import preprocessing
%matplotlib inline 

graph = tf.get_default_graph()

# tf variables
x_ = tf.placeholder(name="input", shape=[None, 1], dtype=np.float32)
y_ = tf.placeholder(name="output", shape=[None, 1], dtype=np.float32)
w = tf.Variable(tf.random_normal([]), name='weight')
b = tf.Variable(tf.random_normal([]), name='bias')

# nonlinear
nonlin_model = tf.add(tf.multiply(tf.tanh(x_), w), b)
nonlin_loss = tf.reduce_mean(tf.pow(nonlin_model - y_, 2), name='cost')
train_step_nonlin = tf.train.GradientDescentOptimizer(0.01).minimize(nonlin_loss)


# pandas data
df_train = pd.read_csv('me_rate.csv', header=None)


liters = df_train.iloc[:, 0].values.reshape(-1, 1)
parrots = df_train.iloc[:, 1].values.reshape(-1, 1)


#model for prediction
mms = preprocessing.MinMaxScaler()
rbf = get_model(path_to_model)


nz = preprocessing.MaxAbsScaler()  # normalization coz tanh
norm_parrots = nz.fit_transform(parrots)
print(norm_parrots)

n_epochs = 20000
train_errors = []
non_train_errors = []
test_errors = []
weights = []
biases = []

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in tqdm.tqdm(range(n_epochs)):

        _, non_train_err, weight, bias = sess.run([train_step_nonlin, nonlin_loss, w, b],
                                      feed_dict={x_: norm_parrots, y_: liters})
        non_train_errors.append(non_train_err)
        weights.append(weight)
        biases.append(bias)


    plt.scatter(norm_parrots, liters, label='actual data')

    plt.plot(norm_parrots, sess.run(nonlin_model, feed_dict={x_: norm_parrots}), c='orange', label='nonlinear (tf)')
    plt.plot(norm_parrots, rbf.predict(mms.fit_transform(parrots)), label='rbf (sklearn)')
    plt.legend()
    plt.show()